## Install the geocoder

In [1]:
!pip install geocoder


## Get the survey data

- [survey](https://docs.google.com/forms/d/1oF7us4ASuFpF6iOhn4g5MnXf4Z8E5EPaIe2mMdPw_cM/edit)
- [data](https://docs.google.com/spreadsheets/d/1qPtvXZJ-IoNXdbVcPxr4buVLUONGsYS7-xNGqPFP144/edit#gid=696092516)


In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSg8WVXVb47Tri6-JLpYZ-rxXUr3iX5HRD23rzK5XmABGpQI3jfSZk9btVTiguQZIgIWAmecSjbXvUM/pub?gid=696092516&single=true&output=csv')
                 

In [17]:
df

,Timestamp,Where do you live? Enter a zipcode.,"If it were up to you, how would you want to work moving forward?",Email Address
0,10/8/2021 11:30:07,90230,hybrid,NaN
1,10/8/2021 13:00:34,91401,hybrid,NaN
2,10/8/2021 13:00:35,91320,100% remote,NaN
3,10/8/2021 13:00:52,91107,100% in person,NaN
4,10/8/2021 13:00:57,90069,100% remote,NaN
5,10/8/2021 13:01:04,92692,100% remote,NaN
6,10/8/2021 13:01:19,90401,hybrid,NaN
7,10/8/2021 13:01:43,91360,hybrid,NaN
8,10/8/2021 13:01:47,90013,hybrid,NaN
9,10/8/2021 13:01:49,90405,hybrid,NaN


## Rename columns

In [5]:
df.columns=['timestamp','zip','work','email']

In [6]:
df

,timestamp,zip,work,email
0,10/8/2021 11:30:07,90230,hybrid,NaN
1,10/8/2021 13:00:34,91401,hybrid,NaN
2,10/8/2021 13:00:35,91320,100% remote,NaN
3,10/8/2021 13:00:52,91107,100% in person,NaN
4,10/8/2021 13:00:57,90069,100% remote,NaN
5,10/8/2021 13:01:04,92692,100% remote,NaN
6,10/8/2021 13:01:19,90401,hybrid,NaN
7,10/8/2021 13:01:43,91360,hybrid,NaN
8,10/8/2021 13:01:47,90013,hybrid,NaN
9,10/8/2021 13:01:49,90405,hybrid,NaN


## Geocode the zipcodes

In [7]:
import geocoder

In [8]:
df['lat'] = ''
df['lon'] = ''

In [9]:
for index, row in df.iterrows():
    g = geocoder.mapbox(row.zip,key='pk.eyJ1IjoiY2h3b2lwcm9qZWN0IiwiYSI6ImNrdTRjZXkwaTRwaWwycXBtY290MjA5d3gifQ.iyENw-MBBY3_9tMPv7frFg')
    print(g.latlng)
    df.loc[index,'lat'] = g.latlng[0]
    df.loc[index,'lon'] = g.latlng[1]
    

[33.98, -118.39]
[34.17, -118.42]
[34.18, -118.89]
[34.15, -118.08]
[34.09, -118.38]
[33.55, -117.67]
[34.01, -118.49]
[34.18, -118.88]
[34.05, -118.25]
[34, -118.48]
[33.68, -117.85]
[34.03, -118.39]
[33.83, -118.35]
[22.06, -159.32]
[33.66, -118]
[34.12, -118.48]


In [10]:
df

,timestamp,zip,work,email,lat,lon
0,10/8/2021 11:30:07,90230,hybrid,NaN,33.98,-118.39
1,10/8/2021 13:00:34,91401,hybrid,NaN,34.17,-118.42
2,10/8/2021 13:00:35,91320,100% remote,NaN,34.18,-118.89
3,10/8/2021 13:00:52,91107,100% in person,NaN,34.15,-118.08
4,10/8/2021 13:00:57,90069,100% remote,NaN,34.09,-118.38
5,10/8/2021 13:01:04,92692,100% remote,NaN,33.55,-117.67
6,10/8/2021 13:01:19,90401,hybrid,NaN,34.01,-118.49
7,10/8/2021 13:01:43,91360,hybrid,NaN,34.18,-118.88
8,10/8/2021 13:01:47,90013,hybrid,NaN,34.05,-118.25
9,10/8/2021 13:01:49,90405,hybrid,NaN,34,-118.48


## Map it!

In [11]:
import folium

In [12]:
from folium.plugins import FloatImage

# image_file = 'image.PNG'

# FloatImage(image_file, bottom=0, left=86).add_to(mymap)

In [13]:
# map setup
m = folium.Map(location = [34.0703726,-118.4453799], tiles="Stamen Terrain", zoom_start=10)

# add radius circles around ucla
radii = [40,30,20,10]
for radius in radii:
    folium.Circle(
        popup=str(radius)+" km radius from UCLA",
        tooltip=str(radius)+" km radius from UCLA",
        radius=radius*1000,
        location=[34.0703726,-118.4453799],
        color="crimson",
        fill=True,
        opacity=0.3,
        fill_opacity=0.05
    ).add_to(m)

# add markers with tooltips colored by desired work mode
for index, row in df.iterrows():
    tooltip = 'I live in zipcode '+str(row.zip)+' and want to work '+row.work
    # color
    if row.work == '100% remote':
        color = 'red'
    elif row.work == 'hybrid':
        color = 'darkblue'
    else:
        color = 'green'
    folium.Marker([row.lat, row.lon], 
                  popup=row.work, 
                  tooltip=tooltip,
                  icon=folium.Icon(color=color,icon='user'),
                 ).add_to(m)
    
FloatImage('https://raw.githubusercontent.com/yohman/interactive-engagement/main/legend3.png',bottom=3,left=2).add_to(m)
# output the map
m

In [14]:
# save the interactive maps as an html file
m.save('index.html')